In [1]:
import sys
sys.path.insert(19, 'litepose-pose-estimation/src')
sys.path.insert(20, 'src/referee_gloves_detector')
import json
import torch
from torchvision import transforms
import lp_config.lp_common_config as cc
from lp_model.lp_litepose import LitePose
from lp_inference.lp_inference import inference, assocEmbedding
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, drawSkeleton
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
from preprocessing import red_filtering, segmentation_and_cropping, equalizing, squaring
import math
from Homography import Homography

In [2]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_big_arch = config_data['path_big_arch']
file_path_csv_keypoints = config_data['path_csv_keypoints']

In [3]:
model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load(file_path_big_arch, map_location=cc.config["device"]))

<All keys matched successfully>

In [4]:
def keypoints_saver(restricted_keypoints):
    d = {'x': [], 'y': [], 'tag':[]}
    df_keypoints = pd.DataFrame(data=d)
    for i in range(len(restricted_keypoints[0])):
        if len(restricted_keypoints[0][i])==0:
            df_temp = pd.DataFrame({'x':np.nan, 'y':np.nan, 'tag':np.nan}, index=[i])
            df_keypoints = pd.concat([df_keypoints, df_temp])
        else:
            print(restricted_keypoints[0][i][0])
            df_temp = pd.DataFrame(restricted_keypoints[0][i][0], index=[i])
            df_keypoints = pd.concat([df_keypoints, df_temp])

    df_keypoints.to_csv(file_path_csv_keypoints, index=False)

In [5]:
# HOMOGRAPHY IMAGE

uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\resources\\bloisi_nao_red_hands.png"
img = Image.open(uri).convert('RGB')
resize = transforms.Resize([224, 224])
img = resize(img)
to_tensor = transforms.ToTensor()
tensor = to_tensor(img)
tensor = tensor.unsqueeze(0)
output, keypoints = inference(model, tensor)
embedding = assocEmbedding(keypoints)
idx = 0
img = drawKeypoints(tensor[idx], keypoints[idx])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
im_pil = Image.fromarray(img)
im_pil.save('src/referee_gloves_detector/resources/src.jpg')
keypoints_saver(['01/01/1970 01:00:00'], [keypoints])
src = cv2.imread("src/referee_gloves_detector/resources/src.jpg")
dst = cv2.imread("src/referee_gloves_detector/resources/dst.jpg")
h = Homography(src,dst)
h._from_detection()
plan_view = cv2.warpPerspective(src, h.H, (dst.shape[1], dst.shape[0]))
cv2.imshow("Reprojected view", plan_view)
cv2.waitKey()
cv2.destroyAllWindows()

TypeError: keypoints_saver() takes 1 positional argument but 2 were given

In [6]:
# HOMOGRAPHY IMAGE AUTOMATICALLY

uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\resources\\bloisi_nao_red_hands.png"
img = Image.open(uri).convert('RGB')
resize = transforms.Resize([224, 224])
img = resize(img)
to_tensor = transforms.ToTensor()
tensor = to_tensor(img)
tensor = tensor.unsqueeze(0)
output, keypoints = inference(model, tensor)
restricted_keypoints = [[keypoints[0][0], keypoints[0][1], keypoints[0][6], keypoints[0][7], keypoints[0][8], keypoints[0][9], keypoints[0][10],keypoints[0][11],keypoints[0][12],keypoints[0][13]]]
embedding = assocEmbedding(keypoints)
idx = 0
img = drawKeypoints(tensor[idx], restricted_keypoints[idx])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
im_pil = Image.fromarray(img)
im_pil.save('src/referee_gloves_detector/resources/src.jpg')
keypoints_saver(restricted_keypoints)
src = cv2.imread("src/referee_gloves_detector/resources/src.jpg")
dst = cv2.imread("src/referee_gloves_detector/resources/dst.jpg")
h = Homography(src,dst)
h._from_detection_automatically()
plan_view = cv2.warpPerspective(src, h.H, (dst.shape[1], dst.shape[0]))
cv2.imshow("Reprojected view", plan_view)
cv2.waitKey()
cv2.destroyAllWindows()

{'x': 125, 'y': 45, 'tag': 0.45596110820770264}
{'x': 65, 'y': 50, 'tag': 0.3447529077529907}
{'x': 116, 'y': 113, 'tag': 0.39876893162727356}
{'x': 93, 'y': 113, 'tag': 0.4468965232372284}
{'x': 129, 'y': 162, 'tag': 0.3563166558742523}
{'x': 86, 'y': 159, 'tag': 0.37395671010017395}
{'x': 146, 'y': 202, 'tag': 0.4609355628490448}
{'x': 93, 'y': 200, 'tag': 0.4463735818862915}
{'x': 91, 'y': 5, 'tag': 0.39669370651245117}
{'x': 94, 'y': 38, 'tag': 0.45942795276641846}
125.0
45.0
65.0
50.0
116.0
113.0
93.0
113.0
129.0
162.0
86.0
159.0
146.0
202.0
93.0
200.0
91.0
5.0
94.0
38.0


C:\Users\stolf\anaconda3\envs\visione-percezione-lite\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[[65, 50], [86, 159], [91, 5], [93, 113], [93, 200], [94, 38], [116, 113], [125, 45], [129, 162], [146, 202]]
[0 2 0 2 1 0 2 0 1 1]
CLUSTER N.  0
   Group  Points_x  Points_y
2      0        91         5
5      0        94        38
7      0       125        45
0      0        65        50
CLUSTER N.  1
   Group  Points_x  Points_y
8      1       129       162
4      1        93       200
9      1       146       202
CLUSTER N.  2
   Group  Points_x  Points_y
3      2        93       113
6      2       116       113
1      2        86       159
[1 0 2 2 1 1 2 0 0 1]
Matrix: A : meanx 103.8, meany 108.7, varx 532.9600000000002, vary 4570.409999999999, sx 0.06125868786374019, sy 0.02091883128172933 
Matrix: B : meanx 165.5, meany 297.7, varx 3888.05, vary 30671.01, sx 0.022680314746437415, sy 0.008075156707140992 
Numero punti campo reale: 10
Numero punti campo virtuale: 10
Number of points in current view :  10
Shape of Matrix M :  (20, 9)
N_model
 [[ 0.06125869  0.         -6.3586518 ]

C:\Users\stolf\anaconda3\envs\visione-percezione-lite\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# HOMOGRAPHY VIDEO

uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\resources\\referee_1.mp4"
cap = cv2.VideoCapture(uri)

# Verifico se il video è stato aperto correttamente
if not cap.isOpened():
    raise Exception("Errore nell'apertura del video")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])
to_tensor = transforms.ToTensor()
ret,frame=cap.read()
count = 0
while ret:

    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante)

    #-----------------------------PREPROCESSING START HERE-----------------------------
    #NORMALIZATION
    normalized_image = cv2.normalize(frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #EQUALIZATION
    equalized_image = equalizing(normalized_image)
    #------------------------------PREPROCESSING END HERE------------------------------
    img = cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB)
    img = squaring(img)

    try:
        # Può dare problemi se gli arriva una immagine con una dimensione mancante,
        # non ho capito il perchè (probabilmente è causata dall'algoritmo di cropping)
        # ma questo può succedere
        im_pil = Image.fromarray(img)
    except:
        print("Shape dell'immagine che farebbe crashare il processo: " + str(img.shape))
        pass
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)

    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        #frame_modified = drawSkeleton(tensor[0], embedding[0])
        frame = drawKeypoints(tensor[0], keypoints[0])
        if count == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image_pil = Image.fromarray(frame, 'RGB')
            image_pil.save('src/referee_gloves_detector/resources/src.jpg')
            keypoints_saver(timestamps, keypoints_vec)
            src = cv2.imread("src/referee_gloves_detector/resources/src.jpg")
            dst = cv2.imread("src/referee_gloves_detector/resources/dst.jpg")
            h = Homography(src,dst)
            h._from_detection()
            count = count + 1
        frame = cv2.warpPerspective(frame, h.H, (dst.shape[1], dst.shape[0]))
        cv2.imshow("Pose estimation", frame)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break

    ret, frame = cap.read()
cap.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))

In [ ]:
# Apro la webcam e salvo al volo i frame e il timestamp

webcam=cv2.VideoCapture(0)

if not webcam.isOpened():
    raise Exception("Errore nell'apertura della webcam")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])
to_tensor = transforms.ToTensor()
ret,frame=webcam.read()
while ret:


    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante)

    frame = cv2.flip(frame, 1)
    #-----------------------------PREPROCESSING START HERE-----------------------------
    #RED FILTERING
    full_mask = red_filtering(frame)
    #SEGMENTATION E CROPPING
    cropped_image = segmentation_and_cropping(frame, full_mask)
    #NORMALIZATION
    normalized_image = cv2.normalize(cropped_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #EQUALIZATION
    equalized_image = equalizing(normalized_image)
    #------------------------------PREPROCESSING END HERE------------------------------
    img = cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB)
    img = squaring(img)

    try:
        # Può dare problemi se gli arriva una immagine con una dimensione mancante,
        # non ho capito il perchè (probabilmente è causata dall'algoritmo di cropping)
        # ma questo può succedere
        im_pil = Image.fromarray(img)
    except:
        print("Shape dell'immagine che farebbe crashare il processo: " + str(img.shape))
        pass
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)

    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        frame_modified = drawSkeleton(tensor[0], embedding[0])
        #frame = drawKeypoints(tensor[0], keypoints[0])
        cv2.imshow("Pose estimation", frame_modified)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break
    ret, frame = webcam.read()
webcam.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))